In [ ]:
%load_ext autoreload
%autoreload 2

# Install API package

Please use the follwing command to install the package:

`pip install -U http://deployments.molcube.com/api/molcube-0.3.0-py3-none-any.whl`

In [ ]:
import molcube as mc

# Obtain API Key

To retrieve the API key for a user, there are two methods available:

Case 1.
1. Authenticate using your MolCube-Builder account credentials (username and password).

Case 2. 
1. Log in to the MolCube-Builder web application and navigate to the Profile page.
2. Generate an API key and copy the key value.
3. Use the copied API key for authentication.

## Additional Note
- The API key does not have an expiration time.
- However, if either method is executed and the API key is regenerated, the previous API key will no longer be valid and cannot be used.

# Authenticate API

In [ ]:
# change the hostname for your API server
molcube = mc.API('localhost', 8000)

# # Case 1: Authenticate with username and password
# username = "username"
# password = "password"
# molcube.authenticate(username=username, password=password)

# Case 2: Authenticate with API key
api_access_key = "api_access_key"
molcube.authenticate(api_token=api_access_key)

# Creating a PDB Reader Project

Let’s walk through how to create a PDB Reader project. The current version supports PDB reading but does not yet allow additional modifications. Expanded modification options will be available in the upcoming release.

The following arguments are available for the method:
* correct_topo (bool): Correct chains and bonds information using distance between each atom. (default: False)
* rename_dupl_atoms (bool): Rename hetero atoms if there are duplicate atom names. (default: True)
* calc_pka (bool): Calculate pKa of protein residues to apply system pH. (defualt: False)

Two force field options are available: `charmmff` and `amberff`. For `amberff`, you can select different force field options. The default selections for `amberff` are as follows:

```
amberOptions = {
    "protein": "FF19SB",
    "dna": "OL15",
    "rna": "OL3",
    "glycan": "GLYCAM_06j",
    "lipid": "Lipid21",
    "water": "OPC"
}
```

Here are all available choices for amberOptions:

* Protein: [FF19SB, FF14SB, FF14SBonlysc]
* DNA: [OL15, BSC1]
* RNA: [OL3, YIL, Shaw]
* Glycan: [GLYCAM_06j]
* Lipid: [Lipid21, Lipid17]
* Water: [OPC, TIP3P, TIP4PEW, TIP4PD]

In [ ]:
# Create a PDB Reader project and fetch PDB from RCSB using PDB ID
pdbreader = molcube.create_pdb_reader_project()
assert pdbreader.create_project(title='test', ff='charmmff')
assert pdbreader.read_pdb_from_rcsb("2hac")

To create a PDB Reader using a custom PDB file, refer to the following example.

After retrieving the default option information using the `model_options()` method, apply the desired options and proceed with modeling. Refer to the `available_info()` method below for guidance on applying options.

In [ ]:
option = pdbreader.model_options()

The `available_info()` method provides information on the options that can be applied to the `option` dictionary in `model_pdb()`.

The output includes the protein chain, terminal, and standalone ligand, while other details are saved in the `available_modification.json` file.

The `option` dictionary follows the structure below for modification options:
- The `ph` option is only applicable if you set `calc_pka` to `True` in the previous step.

```
option = {
    'ph': system_ph: float,
    'protein': {
        chainIndex: bool,
        ...
    },
    'ion': {
        chainIndex: bool,
        ...
    },
    'water': {
        chainIndex: bool,
        ...
    },
    'heme': {
        chainIndex: bool,
        ...
    },
    'necleicAcid': {
        chainIndex: bool,
        ...
    },
    'disulfide': [
        {
            'chainIndex_1': chainIndex,
            'resid_1': resid,
            'chainIndex_2': chainIndex,
            'resid_2': resid,
        },
        ...
    ],
    'mutation': [
        {
            chainIndex: [resid, new_resname],
        },
        ...
    ],
    'terminal': {
        chainIndex: {
            'nter': 'N-terminus',
            'cter': 'C-terminus',
        },
        ...
    },
    'protonation': [
        {
            chainIndex: [resid, patch],
        },
        ...
    ],
    'phosphorylation': [
        {
            chainIndex: [resid, patch],
        },
        ...
    ],
    'missing': [
        {
            'chainIndex': {
                'range': [start_resid, end_resid],
                'discntTerminals': {
                    'nter': 'N-terminus',
                    'cter': 'C-terminus',
                }
            },
        }
        ...
    ],
    'standaloneLigand': [
        {
            'chainIndex': chainIndex,
            'resname': resname,
            'ffType': str, # openff, gaff, xff
            'fast': bool
        }
    ]
}
```

### Example
```
option = {
    'protein': {
        'PROT_A': True,
        'PROT_B': True,
    },
    'disulfide': [{
        'chainIndex_1': 'PROT_A'
        'resid_1': '2'
        'chainIndex_2': 'PROT_B'
        'resid_2': '2'
    }],
    'mutation': [{
        'PROT_A': ['-3', 'ARG']
    }],
    'terminal': {
        "PROT_A": {
            'nter': "NTER",
            'cter': "CTER"
        }
    }
}
```

`assert pdbreader.model_pdb(option)`

In [ ]:
available_info, residue_data = pdbreader.available_info()

In [ ]:
pdbreader.model_pdb(option=option)

The modeled system can be downloaded and visualized either on the user’s computer or through a web interface.

Currently, the API models the PDB structure using default options, without support for user-specified modifications. Additional modification options will be available in a future release.

In [ ]:
pdbreader.download_pdb('system.pdb')

In [ ]:
import nglview as nv
view = nv.show_file('system.pdb')
view

# Creating a Solution Builder Project

Let’s walk through how to create a Solution Builder project. You will need the project ID from a previously created PDB Reader project. For this example, we’ll use the project we just created above.

In [ ]:
pdbreader_project_id = pdbreader.projectId

In [ ]:
sbuilder = molcube.create_solution_builder_project()
assert sbuilder.create_project(title="my_solution_project", pdbreader_project_id=pdbreader_project_id)

We will combine the individual components to build the system for molecular dynamics (MD) simulation. The following arguments are available for the method:

* boxShape(str): System box shape (defulat: cubic)
* margin(float): box margin of Solution System (default: 10).
* ions (str): Type of ions to use (default: "kcl").
* ionConc (float): Ion concentration (default: 0.15).
* neutralize (bool): Whether to neutralize the system (ion-related option) (default: Fasle).
* temperature (float): Temperature for the system in Kelvin (default: 310.0).
* useHmr (bool): Whether to use hydrogen mass repartitioning (default: False).
* mdEngines (list[str]): List of molecular dynamics engines to use (default: ['openmm']).

The available options for ions and mdengine are as follows:

* boxShape: ['cubic', 'truncated_octahedron']
* ions: ['kcl', 'nacl', 'cacl2']
* mdEngines: ['openmm', 'amber', 'gromacs', 'namd', 'genesis', 'desmond']

In [ ]:
mdEngines = ['openmm', 'gromacs']
assert sbuilder.generate_system(mdEngines=mdEngines)

# Creating a Membrane Builder Project

Let’s walk through how to create a Membrane Builder project. You will need the project ID from a previously created PDB Reader project. For this example, we’ll use the project we just created above.

In [ ]:
pdbreader_project_id = pdbreader.projectId

In [ ]:
# Create Membrane Builder project
mbuilder = molcube.create_membrane_builder_project()
assert mbuilder.create_project(title="my_membrane_project",
                               pdbreader_project_id=pdbreader_project_id)

In [ ]:
# Align protein and build membrane
# other options are 'asis', 'principal', 'ppm'
assert mbuilder.align_membrane_protein(option='ppm')

To create a membrane-only system, refer to the following example.

In membrane-only system, the `align_membrane_protein()` method is unnecessary.

The following method generates lipid packing with a specified composition. The composition must be provided as integers. For example, a 9:1 ratio of POPC to cholesterol can be achieved with the example below. Additionally, 20 Å of padding is added to the system for packing.

In [ ]:
# Build lipid packing

# membrane system
box_margin = {
    'margin': 20,
}

# membrane-only system
# box_margin = {
#     'sizeX': 50,
#     'sizeY': 50,
#     'sizeZ': 50,
# }

assert mbuilder.generate_lipid_packing({"POPC": 9, "CHL": 1}, box_margin)

Let's download the dummy packing file and visualize the file.

In [ ]:
mbuilder.download_dummy()
view = nv.show_file('dummy.pdb')
view

In [ ]:
# Replace lipid packing and build system
assert mbuilder.replace_packing()

In [ ]:
mbuilder.download_packing()
view = nv.show_file('packing.pdb')
view

Finally, we will combine the individual components to build the system for molecular dynamics (MD) simulation. The following arguments are available for the method:

* waterMargin(float): water margin of Membrane System (default: 22.5).
* ions (str): Type of ions to use (default: "kcl").
* ionConc (float): Ion concentration (default: 0.15).
* useHmr (bool): Whether to use hydrogen mass repartitioning (default: False).
* mdEngines (list[str]): Molecular dynamics engine to use (default: ['openmm']).

The available options for ions and mdengine are as follows:

* ions: ['kcl', 'nacl', 'cacl2']
* mdEngines: ['openmm', 'amber', 'gromacs', 'namd', 'genesis', 'desmond']

In [ ]:
assert mbuilder.generate_system(ions='kcl', mdEngines=['openmm', 'gromacs'])

In [ ]:
mbuilder.download_system()
view = nv.show_file('system.pdb')
view